# Import Necessary libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import collections
import os
import json

# Defining Global Variables

In [2]:
root_folder = "data"
datatype_folder = "csv"
data_folders = ["buy","communicate","fun","hope","mother","really"]

path = os.path.join("..", root_folder, datatype_folder)

# Extracting only useful data

In [3]:
feature_list = ["leftEye_x", "leftEye_y", "rightEye_x",  "rightEye_y", "nose_x", "nose_y", "leftShoulder_x", "leftShoulder_y", "rightShoulder_x", "rightShoulder_y", "leftElbow_x", "leftElbow_y", "rightElbow_x", "rightElbow_y", "leftWrist_x", "leftWrist_y", "rightWrist_x", "rightWrist_y"]
label_dict = {'buy':0,'communicate':1,'fun':2,'hope':3,'mother':4,'really':5}
X = []
Y = []
for sign in data_folders:
    for file in os.listdir(os.path.join(path, sign)):
        file_path = os.path.join(path, sign, file)
        
        df = pd.read_csv(file_path)[feature_list]
        X.append(df.to_numpy())
        Y.append(label_dict[sign])

# Min and Max Time frames in a sample

In [4]:
min_timeframes = min([len(sign) for sign in X])
max_timeframes = max([len(sign) for sign in X])
print(min_timeframes, max_timeframes)

33 232


# Creating dataframe of same size
---------------
Adding '0' content to videos of smaller size to make them just as long as the longets one. Reducing the sise of larger videos would remove important information from them.

In [5]:
for i in range(len(X)):
    diff = max_timeframes - len(X[i])
    zeros = np.zeros(shape=(diff, len(feature_list)))
    X[i] = np.append(X[i], zeros, axis=0)

X = np.array(X)
Y = np.array(Y)
print(X.shape, Y.shape)

(415, 232, 18) (415,)


# Normalizing data
-------------


In [6]:
for i in range(len(X)):
    x_denom = abs(X[i,:,0] - X[i,:,2])
    y_denom = abs(X[i,:,1] - X[i,:,3])
    x_num = X[i,:,4]
    y_num = X[i,:,5]

    for col in range(6, len(feature_list), 2):
        X[i,:,col] = np.divide(X[i,:,col]-x_num, x_denom, out=np.zeros((max_timeframes, )), where=x_denom!=0)
    
    for col in range(7, len(feature_list), 2):
        X[i,:,col] = np.divide(X[i,:,col]-y_num, y_denom, out=np.zeros((max_timeframes, )), where=y_denom!=0)

# Saving Numpy Array
-----------------
saving the entire normalized numpy data and the label so that we dont run it again and again. We are only saving features that we need, so columns of nose and eye data have been removed

In [7]:
np.save(os.path.join("..", "IPD", "data_numpy.npy"), X[:,:,6:]) #shape-> (415, 232, 12) 
np.save(os.path.join("..", "IPD", "label_numpy.npy"), Y) #shape-> (415, )